In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import glob
import os
import time
import feather
from sktime.classification.feature_based import RandomIntervalClassifier
from sktime.datasets import load_from_tsfile_to_dataframe
sns.set(style="whitegrid")

In [ ]:
from RIC_transformers import *

--------------------------------------

### LOAD DATA

In [ ]:
# load training data
X_train, y_train = load_from_tsfile_to_dataframe('data/OliveOil_TRAIN.ts')

# load test data
X_test, y_test = load_from_tsfile_to_dataframe('data/OliveOil_TEST.ts')

In [ ]:
#datasets = ['Adiac', 'Fish', 'OliveOil', 'SwedishLeaf']
datasets = [Fish', 'OliveOil', 'SwedishLeaf']


# --> reduce runtime, number of intervals < 31
nbrIntervals = [30, 24, 12, 27]

classifiers = ['RIC', 
               'RIC_catch22', 
               'RIC_kats',
               'RIC_tsfel',
               'RIC_tsfresh']

transformers = [None, Catch22(), Kats(), TSFel(), TSFresh()]
scores = np.zeros((len(transformers), len(datasets)))

--------------------------------------

In [ ]:
def calcScore(X_train, y_train, X_test, y_test, nbrIntervals, transformer=None):
    
    classifier = RandomIntervalClassifier(n_intervals=nbrIntervals, interval_transformers=transformer, random_state=2)
    
    #start = time.time()
    classifier.fit(X_train, y_train)
    #end = time.time()

    #t0 = time.time()
    score = classifier.score(X_test, y_test)
    #t1 = time.time()

    #print('Fitting time: ', end-start)
    #print('Prediction time: ', t1-t0)
    
    return score
    

In [ ]:
datasets = ['Adiac', 'Fish', 'OliveOil', 'SwedishLeaf']
for i in [0]:
    
    dataset   = datasets[i]
    intervals = nbrIntervals[i]
    print(dataset)
    
    # load training data
    X_train, y_train = load_from_tsfile_to_dataframe('data/' + dataset + '_TRAIN.ts')

    # load test data
    X_test, y_test = load_from_tsfile_to_dataframe('data/' + dataset + '_TEST.ts')
    
    j = 1
    transformer = transformers[j]
    scores[j,i] = calcScore(X_train, y_train, X_test, y_test, intervals, transformer)

In [ ]:
scores

--------------------------------------

In [ ]:
datasets = ['Adiac', 'Fish', 'OliveOil', 'SwedishLeaf']
for i in range(2,4):
    
    dataset = datasets[i]
    print(dataset)
    
    # load training data
    X_train, y_train = load_from_tsfile_to_dataframe('data/' + dataset + '_TRAIN.ts')

    # load test data
    X_test, y_test = load_from_tsfile_to_dataframe('data/' + dataset + '_TEST.ts')
    
    j = 4
    scores[j,i] = calcScore(X_train, y_train, X_test, y_test, TSFresh())

In [ ]:
scores = np.array([[0.77749361, 0.92571429, 0.93333333, 0.928     ],
       [0.76726343, 0.91428571, 0.9       , 0.912     ],
       [0.        , 0.        , 0.86666667, 0.        ],
       [0.78005115, 0.        , 0.9       , 0.        ],
       [0.        , 0.        , 0.9       , 0.        ]])

In [ ]:
names = ['RIC', 'RIC_catch22', 'RIC_kats', 'RIC_tsfel', 'RIC_tsfresh']
scores = []

def calcScore(transformer=None, X_train, y_train, X_test, y_test):
    
    classifier = RandomIntervalClassifier(n_intervals=12, interval_transformers=transformer, random_state=2)
    
    start = time.time()
    classifier.fit(X_train, y_train)
    end = time.time()
    

    t0 = time.time()
    scores.append(classifier.score(X_test, y_test))
    t1 = time.time()

    #print('Fitting time: ', end-start)
    #print('Prediction time: ', t1-t0)
    

--------------------------------------

ORIGINAL

In [ ]:
calcScore()

CATCH22

In [ ]:
calcScore(transformer=Catch22())

KATS

In [ ]:
calcScore(transformer=Kats())

TSFEL

In [ ]:
calcScore(transformer=TSFel())

TSFRESH

In [ ]:
calcScore(transformer=TSFresh())

--------------------------------------

### Plot

In [ ]:
# row corresponds to classifier, col to dataset
accuracy = pd.DataFrame(scores, columns=datasets)
accuracy

In [ ]:
scores

In [ ]:
def plotPerDataset(axes, y_pos, nbrPlots, df):
    
    for c in range(nbrPlots):
        
        axes[c].set_title(df.columns[c])
        axes[c].set_xlabel('Classifier')
        axes[c].set_ylabel('Accuracy')
        
        # Create bars
        axes[c].bar(y_pos, df.iloc[:, c], color='maroon')
        
        #
        axes[c].set_xticks(y_pos, classifiers, rotation='vertical')
    
    
def plotPerClassifier(axes, y_pos, nbrPlots, df):
    
    for c in range(nbrPlots):
        
        axes[c].set_title(classifiers[c])
        axes[c].set_xlabel('Dataset')
        axes[c].set_ylabel('Accuracy')
        
        # Create bars
        axes[c].bar(y_pos, df.iloc[c], color='maroon')
        
        #
        axes[c].set_xticks(y_pos, datasets, rotation='vertical') 
    

def plotAccuracy(df, perDataset = True):
    
    cols = 3
    
    if perDataset:
        nbrPlots = len(df.columns)
        y_pos    = np.arange(len(df))
    else:
        nbrPlots = len(df)
        y_pos    = np.arange(len(df.columns))
    
    rows = int(np.ceil(nbrPlots/cols))
    
    fig, ax = plt.subplots(rows, cols, figsize=(30, 30))
    # using padding
    fig.tight_layout(pad=14.0)
    
    axes = ax.flatten()
    
    if perDataset:
        plotPerDataset(axes, y_pos, nbrPlots, df)
    else:
        plotPerClassifier(axes, y_pos, nbrPlots, df)
    
     

In [ ]:
plotAccuracy(accuracy)

In [ ]:
plotAccuracy(accuracy, False)

In [ ]:
def plotAccuracy(l1, l2, title):
    
    y_pos = np.arange(len(l1))
    
    # Create bars
    plt.bar(y_pos, l1, color='darkslategrey')
    
    # Create names on the x-axis
    plt.xticks(y_pos, l2,rotation='vertical')

    plt.xlabel('Method')
    plt.ylabel('Accuracy')
    plt.title(title)
    plt.show()
    
plotAccuracy(scores, names, 'RandomIntervalClassifier')